*This product uses the FRED® API but is not endorsed or certified by the Federal Reserve Bank of St. Louis.*


##### I've used this Jupyter notebook for DBT tasks and have already pushed initial code to GitHub's main branch. Next, I'll create branches for each phase: converting pandas code to Apache Spark for scalability, containerizing with Docker, setting up Airflow for orchestration, and integrating SODA-core for data quality. It's worth noting that DBT automatically handles documentation as part of its functionality, aiding in maintaining clear and up-to-date project documentation. For the UI, options will include pushing gold tables to AWS S3, an on-prem location via ODBC, or exporting them to Excel, each refreshed daily. At this UI stage, secured connections and data encryption will be established to ensure data security. This approach ensures an organized, efficient, and secure development process.

In [1]:
## Add index categories as a user input
## Add FRED API Key as an input

fred_series_ids = [
    "DCOILWTICO", "GASREGW", "DPROPANEMBTX", "CPIAUCSL", "PPIACO", "PCE", "CPILFESL", "UMCSENT", "PCEPILFE", "MICH",
    "PAYEMS", "UNRATE", "CES0500000003", "CIVPART", "JTSJOL", "ICSA", "ADPWNUSNERSA", "U6RATE", "AWHNONAG", "FEDFUNDS", 
    "DFEDTAR", "IOER", "DISCOUNT", "DGS10", "SOFR", "PRIME", "TB3MS", "CPFF", "CSUSHPINSA", "HSN1F", "PERMIT", "HOUST", 
    "USSTHPI", "EXHOSLUSM495S", "MORTGAGE30US", "RRVRUSQ156N", "RHORUSQ156N", "HOSINVUSM495N"
]

aaaa00001 =  ['DCOILWTICO', 'GASREGW', 'DPROPANEMBTX']
aaaa00002 =  ['CPIAUCSL', 'PPIACO', 'PCE', 'CPILFESL', 'UMCSENT', 'PCEPILFE', 'MICH']
aaaa00003 =  ['PAYEMS', 'UNRATE', 'CES0500000003', 'CIVPART', 'JTSJOL', 'ICSA', 'ADPWNUSNERSA', 'U6RATE', 'AWHNONAG']
aaaa00004 =  ['FEDFUNDS', 'DFEDTAR', 'IOER', 'DISCOUNT', 'DGS10', 'SOFR', 'PRIME', 'TB3MS', 'CPFF']
aaaa00005 =  ['CSUSHPINSA', 'HSN1F', 'PERMIT', 'HOUST', 'USSTHPI', 'EXHOSLUSM495S', 'MORTGAGE30US', 'RRVRUSQ156N', 'RHORUSQ156N', 'HOSINVUSM495N']

index_categories = {
    'energy': aaaa00001,
    'pricing': aaaa00002,
    'employment': aaaa00003,
    'federal_funds': aaaa00004,
    'housing': aaaa00005
}

number_of_years = 30

target_directory = "/Users/jeffreyblack/Projects/FREDDataProject/fred_economic_data/models"


In [2]:
from scripts import fetch
from scripts import save_to_duckdb
from scripts import bronze_layer
from scripts import silver_layer_unionall
from scripts import silver_layer_binaryconversion
from scripts import silver_layer_setdatatypes
from scripts import gold_layer
from scripts import ref_doc_gen 

dataframes, metadata_df, category_df = fetch.main(fred_series_ids, index_categories)
save_to_duckdb.main(fred_series_ids, dataframes, metadata_df, category_df)
bronze_layer.main(fred_series_ids, target_directory)
silver_layer_unionall.main(fred_series_ids, target_directory)
silver_layer_binaryconversion.main(target_directory)
silver_layer_setdatatypes.main(target_directory)
gold_layer.main(target_directory)
ref_doc_gen.main(fred_series_ids, target_directory) 


SQL file created: /Users/jeffreyblack/Projects/FREDDataProject/fred_economic_data/models/bronze/bronze_FactDCOILWTICO.sql
SQL file created: /Users/jeffreyblack/Projects/FREDDataProject/fred_economic_data/models/bronze/bronze_FactGASREGW.sql
SQL file created: /Users/jeffreyblack/Projects/FREDDataProject/fred_economic_data/models/bronze/bronze_FactDPROPANEMBTX.sql
SQL file created: /Users/jeffreyblack/Projects/FREDDataProject/fred_economic_data/models/bronze/bronze_FactCPIAUCSL.sql
SQL file created: /Users/jeffreyblack/Projects/FREDDataProject/fred_economic_data/models/bronze/bronze_FactPPIACO.sql
SQL file created: /Users/jeffreyblack/Projects/FREDDataProject/fred_economic_data/models/bronze/bronze_FactPCE.sql
SQL file created: /Users/jeffreyblack/Projects/FREDDataProject/fred_economic_data/models/bronze/bronze_FactCPILFESL.sql
SQL file created: /Users/jeffreyblack/Projects/FREDDataProject/fred_economic_data/models/bronze/bronze_FactUMCSENT.sql
SQL file created: /Users/jeffreyblack/Proje

In [3]:
import duckdb
import pandas as pd

sql_query = '''
SELECT 
SeriesID,
RealTimeStart, 
RealTimeEnd,
Name, 
ObservationStart,
ObservationEnd, 
Frequency, 
FrequencyAbbr, 
Units,
UnitsAbbr, 
SeasAdj,
SeasAdjAbbr,
CASE 
    WHEN SeasAdjAbbr = 'NSA' THEN False 
    WHEN SeasAdjAbbr = 'SA' THEN True 
    WHEN SeasAdjAbbr = 'SAAR' THEN True 
    ELSE NULL 
END AS SeasAdjBool
FROM bronze_DimSeries
'''

with duckdb.connect('data/fred.db') as con:
    display(con.sql(sql_query).df())

,SeriesID,RealTimeStart,RealTimeEnd,Name,ObservationStart,ObservationEnd,Frequency,FrequencyAbbr,Units,UnitsAbbr,SeasAdj,SeasAdjAbbr,SeasAdjBool
0,DCOILWTICO,2024-03-25,2024-03-25,Crude Oil Prices: West Texas Intermediate (WTI...,1986-01-02,2024-03-18,Daily,D,Dollars per Barrel,$ per Barrel,Not Seasonally Adjusted,NSA,False
1,GASREGW,2024-03-25,2024-03-25,US Regular All Formulations Gas Price,1990-08-20,2024-03-18,"Weekly, Ending Monday",W,Dollars per Gallon,$ per Gallon,Not Seasonally Adjusted,NSA,False
2,DPROPANEMBTX,2024-03-25,2024-03-25,"Propane Prices: Mont Belvieu, Texas",1992-07-09,2024-03-18,Daily,D,Dollars per Gallon,$ per Gallon,Not Seasonally Adjusted,NSA,False
3,CPIAUCSL,2024-03-25,2024-03-25,Consumer Price Index for All Urban Consumers: ...,1947-01-01,2024-02-01,Monthly,M,Index 1982-1984=100,Index 1982-1984=100,Seasonally Adjusted,SA,True
4,PPIACO,2024-03-25,2024-03-25,Producer Price Index by Commodity: All Commodi...,1913-01-01,2024-02-01,Monthly,M,Index 1982=100,Index 1982=100,Not Seasonally Adjusted,NSA,False
5,PCE,2024-03-25,2024-03-25,Personal Consumption Expenditures,1959-01-01,2024-01-01,Monthly,M,Billions of Dollars,Bil. of $,Seasonally Adjusted Annual Rate,SAAR,True
6,CPILFESL,2024-03-25,2024-03-25,Consumer Price Index for All Urban Consumers: ...,1957-01-01,2024-02-01,Monthly,M,Index 1982-1984=100,Index 1982-1984=100,Seasonally Adjusted,SA,True
7,UMCSENT,2024-03-25,2024-03-25,University of Michigan: Consumer Sentiment,1952-11-01,2024-01-01,Monthly,M,Index 1966:Q1=100,Index 1966:Q1=100,Not Seasonally Adjusted,NSA,False
8,PCEPILFE,2024-03-25,2024-03-25,Personal Consumption Expenditures Excluding Fo...,1959-01-01,2024-01-01,Monthly,M,Index 2017=100,Index 2017=100,Seasonally Adjusted,SA,True
9,MICH,2024-03-25,2024-03-25,University of Michigan: Inflation Expectation,1978-01-01,2024-01-01,Monthly,M,Percent,%,Not Seasonally Adjusted,NSA,False
